In [1]:
pip install networkx beautifulsoup4 requests

  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached certifi-2024.2.2-py3-none-any.whl.metadata (2.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 1.4 MB/s eta 0:00:0000:0100:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.9/147.9 kB 2.4 MB/s eta 0:00:00a 0:00:01
Using cached requests-2.31.0-py3-none-any.whl (62 kB)
Using cached certifi-2024.2.2-py3-none-any.whl (163 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.1/119.1 kB 2.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 3.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pandas torch torchvision torch-geometric


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 1.1 MB/s eta 0:00:00a 0:00:01
  Using cached Jinja2-3.1.3-py3-none-any.whl.metadata (3.3 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 MB 15.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.1 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 391.9/391.9 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 11.6 MB/s eta 0:00:00
Using cached Jinja2-3.1.3-py3-none-any.whl (133 kB)
Using cached sympy-1.12-py3-none-any.whl (5.7 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 5.5 MB/s eta 0:00:00
Using cac

In [1]:
import pandas as pd

dataset = pd.read_csv('/Users/rollie/Documents/URL_Phishing_Detection/notebooks/filtered_file.csv')
import networkx as nx
import numpy as np
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import requests
from bs4 import BeautifulSoup

#hàm trích xuất tất cả các hyperlink từ một URL 
def fetch_hyperlinks(url):
     
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        links = [link.get('href') for link in soup.find_all('a') if link.get('href')]
        return links
    except Exception as e:
        print(f"Error fetching {url}: {e}")
        return []

def build_graph(dataset):
    G = nx.DiGraph()
    url_to_index = {}
    features = []
    labels = []
    index = 0
    for _, row in dataset.iterrows():
        url = row['URL']
        if url not in url_to_index:
            G.add_node(url)
            url_to_index[url] = index
            # lấyy đặc trưng trongg tập dataset: 'TLD', 'URLLength', 'IsDomainIP', 'NoOfSubDomain', 'NoOfObfuscatedChar', 'IsHTTPS', 'NoOfDegitsInURL', 'NoOfEqualsInURL', 'NoOfQMarkInURL','NoOfAmpersandInURL'
            features.append([row['TLD'], row['URLLength'], row['IsDomainIP'], row['NoOfSubDomain'], row['NoOfObfuscatedChar'], row['IsHTTPS'], row['NoOfDegitsInURL'], row['NoOfEqualsInURL'], row['NoOfQMarkInURL'], row['NoOfAmpersandInURL']])
            labels.append(row['label'])
            index += 1
        hyperlinks = fetch_hyperlinks(url)
        for link in hyperlinks:
            if link in url_to_index:  # Chỉ thêm cạnh nếu hyperlink đã là một node
                G.add_edge(url, link)

    features = np.array(features)
    labels = np.array(labels)
    return G, features, labels

graph, features, labels = build_graph(dataset)

#cb dữ liệu Pytorch
x = torch.tensor(features, dtype=torch.float)
# Giả sử graph.edges() trả về danh sách các cạnh dưới dạng tuple của chuỗi (ví dụ: [('1', '2'), ('3', '4')])
# Chúng ta cần chuyển đổi chúng thành số nguyên
edge_index = torch.tensor([[int(node) for node in edge] for edge in graph.edges()], dtype=torch.long).t().contiguous()

#edge_index = torch.tensor([list(edge) for edge in graph.edges()], dtype=torch.long).t().contiguous()
y = torch.tensor(labels, dtype=torch.long)

data = Data(x=x, edge_index=edge_index, y=y)

class GCN(torch.nn.Module):
    def __init__(self, num_features, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_features, 16)
        self.conv2 = GCNConv(16, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = torch.relu(self.conv1(x, edge_index))
        x = torch.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return x

model = GCN(num_features=2, num_classes=len(set(labels))) # set 2 lớp GCN
#huấn luyện
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])  # Giả sử bạn đã tạo một mặt nạ huấn luyện
    loss.backward()
    optimizer.step()
    return loss.item()

for epoch in range(200):  # Số lượng epoch huấn luyện
    loss = train()
    print(f'Epoch {epoch}, Loss: {loss}')



KeyboardInterrupt: 